# ANALIZA SENTYMENTU - 4 etykiety

In [141]:
# Importy
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, DataCollatorWithPadding, TrainingArguments, Trainer
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset
import datasets
import accelerate
from imblearn.over_sampling import SMOTE

In [3]:
df = pd.read_csv("reviews_eda1b.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962 entries, 0 to 5961
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Link                5962 non-null   object 
 1   Autor               5962 non-null   object 
 2   Tytuł               5962 non-null   object 
 3   Data_recenzji       5962 non-null   object 
 4   Treść_recenzji      5962 non-null   object 
 5   Średnia_ocen        5962 non-null   float64
 6   Liczba_ocen         5962 non-null   int64  
 7   Liczba_czytelników  5962 non-null   int64  
 8   Rok_recenzji        5962 non-null   int64  
 9   sentyment           5962 non-null   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 465.9+ KB


In [5]:
df.columns

Index(['Link', 'Autor', 'Tytuł', 'Data_recenzji', 'Treść_recenzji',
       'Średnia_ocen', 'Liczba_ocen', 'Liczba_czytelników', 'Rok_recenzji',
       'sentyment'],
      dtype='object')

# Pierwszy model, który przetestuje to będzie RandomForestClassifier
# W związku z tym, że mam wyraźny problem niezrównoważonych klas, będe musiała zastosować technike oversamplingu.
# Nie wiem czy duplikowanie danych to będzie najlepsze rozwiązanie. 
# Najpierw więc zastosuję SMOTE (Synthetic Minority Over-sampling Technique)

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [91]:
# Przygotowuję zbiory: treningowy, walidacyjny i testowy
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

In [92]:
# Tworzę macierz TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train = vectorizer.fit_transform(train_data["tokenized_reviews"])
X_val = vectorizer.transform(val_data["tokenized_reviews"])
X_test = vectorizer.transform(test_data["tokenized_reviews"])

In [93]:
# Etykiety
y_train = train_data["sentyment"]
y_val = val_data["sentyment"]
y_test = test_data["sentyment"]

In [66]:
# Podejście 1 - model RandomForestClassifier
rf_model_base = RandomForestClassifier(random_state=42)
rf_model_base.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [67]:
# Teraz będę oceniać modelu bazowego na zestawie walidacyjnym
y_val_pred_base = rf_model_base.predict(X_val)
print("Classification Report dla modelu bazowego:\n", classification_report(y_val, y_val_pred_base))


Classification Report dla modelu bazowego:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.00      0.00      0.00       185
           2       0.63      1.00      0.77       744
           3       0.62      0.02      0.04       225

    accuracy                           0.63      1193
   macro avg       0.31      0.25      0.20      1193
weighted avg       0.51      0.63      0.49      1193



c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

# W związku z dość kiepskimi wynikami, teraz zastosuję SMOTE

In [68]:
# SMOTE do zestawu treningowego
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [70]:
# Model z zastosowaniem SMOTE
rf_model_resampled = RandomForestClassifier(random_state=42)
rf_model_resampled.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [71]:

# Ocena modelu z zastosowaniem SMOTE na zestawie walidacyjnym
y_val_pred_resampled = rf_model_resampled.predict(X_val)
print("Classification Report dla modelu z zastosowaniem SMOTE:\n", classification_report(y_val, y_val_pred_resampled))

Classification Report dla modelu z zastosowaniem SMOTE:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.44      0.09      0.15       185
           2       0.65      0.96      0.77       744
           3       0.41      0.11      0.17       225

    accuracy                           0.63      1193
   macro avg       0.37      0.29      0.27      1193
weighted avg       0.55      0.63      0.54      1193



c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

# Model XGBoost

# base

In [81]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [84]:
# Utworzenie i trening modelu XGBoost
xgb_model_base = xgb.XGBClassifier()
xgb_model_base.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [85]:
# Ocena modelu na danych walidacyjnych
y_pred_xgb = xgb_model_base.predict(X_test)
print("Classification Report XGBoost:")
print(classification_report(y_test, y_pred_xgb))

Classification Report XGBoost:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.54      0.18      0.27       184
           2       0.69      0.96      0.80       750
           3       0.58      0.23      0.33       223

    accuracy                           0.67      1193
   macro avg       0.45      0.34      0.35      1193
weighted avg       0.62      0.67      0.61      1193



# SMOTE

In [86]:
# Utworzenie i trening modelu XGBoost na danych po SMOTE
xgb_model_smote = xgb.XGBClassifier()
xgb_model_smote.fit(X_train_resampled, y_train_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [87]:
# Ocena modelu XGBoost po SMOTE na danych testowych
y_pred_xgb_smote = xgb_model_smote.predict(X_val)
print("Classification Report XGBoost with SMOTE:")
print(classification_report(y_test, y_pred_xgb_smote))

Classification Report XGBoost with SMOTE:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.17      0.08      0.11       184
           2       0.63      0.83      0.72       750
           3       0.22      0.12      0.16       223

    accuracy                           0.55      1193
   macro avg       0.25      0.26      0.25      1193
weighted avg       0.46      0.55      0.50      1193



# Teraz LSTM

In [127]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer

In [108]:
# Dzielę dane na zbiory train, val, test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)


In [109]:
# Tworzę macierz TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train = vectorizer.fit_transform(train_data["tokenized_reviews"])
X_val = vectorizer.transform(val_data["tokenized_reviews"])
X_test = vectorizer.transform(test_data["tokenized_reviews"])

In [124]:
# Przekształcam macierze rzadkie na gęste przed porządkowaniem indeksów tf.sparse.reorder
X_train_dense = X_train.toarray()
X_val_dense = X_val.toarray()
X_test_dense = X_test.toarray()

In [125]:
# Tworzę SparseTensor z gęstych macierzy
X_train_sparse = tf.sparse.reorder(tf.sparse.from_dense(X_train_dense))
X_val_sparse = tf.sparse.reorder(tf.sparse.from_dense(X_val_dense))
X_test_sparse = tf.sparse.reorder(tf.sparse.from_dense(X_test_dense))

In [110]:
# Etykiety
y_train = train_data["sentyment"]
y_val = val_data["sentyment"]
y_test = test_data["sentyment"]

In [114]:
# Definicja parametrów
max_words = 5000  # maksymalna liczba unikalnych słów
embedding_dim = 128  # wymiar przestrzeni osadzeń


In [133]:
# Budowa modelu
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=None))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(GlobalAveragePooling1D())  # GlobalAveragePooling1D zamiast Flatten
model.add(Dense(4, activation='softmax'))  # 4 klasy sentymentu

In [134]:
# Kompilacja modelu
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [131]:
# Wczesne zatrzymywanie
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [135]:
# Trenowanie modelu
history = model.fit(X_train_sparse, y_train, epochs=10, batch_size=64, validation_data=(X_val_sparse, y_val), callbacks=[early_stopping])


Epoch 1/10


56/56 [==============================] - 4773s 86s/step - loss: 1.0428 - accuracy: 0.6337 - val_loss: 1.0142 - val_accuracy: 0.6236
Epoch 2/10
56/56 [==============================] - 5034s 90s/step - loss: 0.9864 - accuracy: 0.6415 - val_loss: 1.0123 - val_accuracy: 0.6236
Epoch 3/10
56/56 [==============================] - 5045s 90s/step - loss: 0.9868 - accuracy: 0.6415 - val_loss: 1.0139 - val_accuracy: 0.6236
Epoch 4/10
56/56 [==============================] - 5146s 92s/step - loss: 0.9877 - accuracy: 0.6415 - val_loss: 1.0113 - val_accuracy: 0.6236
Epoch 5/10
56/56 [==============================] - 5169s 92s/step - loss: 0.9884 - accuracy: 0.6415 - val_loss: 1.0203 - val_accuracy: 0.6236
Epoch 6/10
56/56 [==============================] - 5202s 93s/step - loss: 0.9891 - accuracy: 0.6415 - val_loss: 1.0110 - val_accuracy: 0.6236
Epoch 7/10
56/56 [==============================] - 6987s 125s/step - loss: 0.9865 - accuracy: 0.6415 - val_loss: 1.0120 - val_accuracy: 0.6

In [140]:
# Ocena modelu
accuracy = model.evaluate(X_test_sparse, y_test)[1]
print(f'Test Accuracy: {accuracy}')

38/38 [==============================] - 19s 493ms/step - loss: 1.0002 - accuracy: 0.6287
Test Accuracy: 0.6286672353744507
